In [1]:
from linares_plot import *
import sys, os
path_tools = 'C:\\Users\\David\\Documents\\GitHub\\encoding\\scripts\\wm_representation\\functions\\IEM\\tools'
sys.path.insert(1, path_tools)
from tiemcourse_interference import *
from scipy import stats
from scipy.stats import norm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_excel('C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_trainT_decoded_angle.xlsx')
len(df)

89856

In [4]:
def err_deg(a1,ref):
    ### Calculate the error ref-a1 in an efficient way in the circular space
    ### it uses complex numbers!
    ### Input in degrees (0-360)
    a1=np.radians(a1)
    ref=np.radians(ref)
    err = np.angle(np.exp(1j*ref)/np.exp(1j*(a1) ), deg=True) 
    err=round(err, 2)
    return err

In [5]:
error_ = [err_deg(df.iloc[i].decoded_angle, df.iloc[i].target_centered) for i in range(len(df)) ]
distr_distance = [err_deg(df.iloc[i].distractor_centered, df.iloc[i].target_centered) for i in range(len(df)) ]

df['error'] = error_
df['error_abs'] = abs(np.array(error_))
df['distractor_distance'] = distr_distance
df['distractor_distance_abs'] = abs(np.array(distr_distance))

df['sign_dist'] = df.distractor_distance<0
df['sign_dist'] = df['sign_dist'].replace([True, False], ['ccw', 'cw']) #position of the distractor relative to the target

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  import sys


In [6]:
df_beh = pd.read_excel('C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_trainT_decoded_angle_related_behaviour.xlsx')

In [7]:
df_beh.head()

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
0,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,d001,10,1,10_1,0,visual,1_0.2
1,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,d001,10,1,10_1,1,visual,1_0.2
2,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,d001,10,1,10_1,2,visual,1_0.2
3,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,d001,10,1,10_1,3,visual,1_0.2
4,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,d001,10,1,10_1,4,visual,1_0.2


In [8]:
len(df_beh)

5964

In [13]:
df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ].new_index.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], dtype=int64)

In [18]:
df_example = df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ]

In [32]:
df_example

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5046,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,l001,1,1,1_1,0,visual,1_0.2
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2
5048,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,l001,1,1,1_1,2,visual,1_0.2
5049,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,l001,1,1,1_1,3,visual,1_0.2
5050,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,l001,1,1,1_1,4,visual,1_0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,2,3,0.2,11.8,334,228,69,142.0,214.9,40.7,...,142,228,214.9,l001,4,4,4_4,75,visual,1_0.2
5122,6,1,0.2,11.8,299,117,37,316.5,145.4,210.0,...,210,299,316.5,l001,4,4,4_4,76,visual,1_0.2
5123,8,3,0.2,11.8,334,123,62,233.0,110.0,39.1,...,233,123,110.0,l001,4,4,4_4,77,visual,1_0.2
5124,18,3,0.2,11.8,333,157,53,218.0,138.8,32.0,...,218,157,138.8,l001,4,4,4_4,78,visual,1_0.2


In [19]:
df_example.columns

Index(['Unnamed: 0', 'type', 'delay1', 'delay2', 'T', 'NT1', 'NT2', 'Dist',
       'Dist_NT1', 'Dist_NT2', 'distance_T_dist', 'cue', 'order', 'orient',
       'horiz_vertical', 'A_R', 'A_err', 'Abs_angle_error',
       'Error_interference', 'A_DC', 'A_DC_dist', 'Q_DC', 'A_DF', 'A_DF_dist',
       'Q_DF', 'A_DVF', 'Q_DVF', 'A_DVF_dist', 'Q_DVF_dist',
       'presentation_att_cue_time', 'presentation_target_time',
       'presentation_dist_time', 'presentation_probe_time', 'R_T',
       'trial_time', 'disp_time', 'T_alone', 'dist_alone', 'T_close',
       'dist_close', 'subject', 'session', 'run', 'session_run', 'new_index',
       'brain_region', 'condition'],
      dtype='object')

In [35]:
sum(df_example['T'].values == df_example['T_alone'].values)

41

In [ ]:
df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ].new_index.unique()

In [37]:
df.head()

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
0,0,190.770087,180,T,False,164.0,False,Dist,0,0.0,d001,visual,1_0.2,-10.77,10.77,16.0,16.0,cw
1,1,165.634003,180,NT1,False,153.7,False,Dist_NT1,0,0.0,d001,visual,1_0.2,14.37,14.37,26.3,26.3,cw
2,2,178.169130,180,NT2,True,283.0,True,Dist_NT2,0,0.0,d001,visual,1_0.2,1.83,1.83,-103.0,103.0,ccw
3,3,170.110002,180,T,False,161.5,False,Dist,4,0.0,d001,visual,1_0.2,9.89,9.89,18.5,18.5,cw
4,4,212.278508,180,NT1,False,153.5,False,Dist_NT1,4,0.0,d001,visual,1_0.2,-32.28,32.28,26.5,26.5,cw


In [64]:
df.loc[(df['subject']=='r001') & (df['condition']=='2_7') & (df['brain_region']=='visual') ]

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
49296,0,199.468705,180,T,True,297.0,True,Dist,1,0.000,r001,visual,2_7,-19.47,19.47,-117.0,117.0,ccw
49297,1,161.891897,180,NT1,False,161.2,False,Dist_NT1,1,0.000,r001,visual,2_7,18.11,18.11,18.8,18.8,cw
49298,2,209.152766,180,NT2,False,150.5,False,Dist_NT2,1,0.000,r001,visual,2_7,-29.15,29.15,29.5,29.5,cw
49299,3,180.564141,180,T,True,346.0,True,Dist,3,0.000,r001,visual,2_7,-0.56,0.56,-166.0,166.0,ccw
49300,4,185.556095,180,NT1,False,164.6,False,Dist_NT1,3,0.000,r001,visual,2_7,-5.56,5.56,15.4,15.4,cw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50491,1195,188.638891,180,NT1,False,197.9,False,Dist_NT1,16,35.025,r001,visual,2_7,-8.64,8.64,-17.9,17.9,ccw
50492,1196,NaN,180,NT2,False,205.4,False,Dist_NT2,16,35.025,r001,visual,2_7,NaN,NaN,-25.4,25.4,ccw
50493,1197,198.634002,180,T,False,192.4,False,Dist,26,35.025,r001,visual,2_7,-18.63,18.63,-12.4,12.4,ccw
50494,1198,NaN,180,NT1,False,206.9,False,Dist_NT1,26,35.025,r001,visual,2_7,NaN,NaN,-26.9,26.9,ccw


In [65]:
df.loc[(df['subject']=='n001') & (df['condition']=='2_7') & (df['brain_region']=='visual') ]

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
19008,0,183.644174,180,T,True,297.0,True,Dist,1,0.000,n001,visual,2_7,-3.64,3.64,-117.0,117.0,ccw
19009,1,171.542720,180,NT1,False,161.2,False,Dist_NT1,1,0.000,n001,visual,2_7,8.46,8.46,18.8,18.8,cw
19010,2,200.863046,180,NT2,False,150.5,False,Dist_NT2,1,0.000,n001,visual,2_7,-20.86,20.86,29.5,29.5,cw
19011,3,178.094140,180,T,True,346.0,True,Dist,3,0.000,n001,visual,2_7,1.91,1.91,-166.0,166.0,ccw
19012,4,150.759727,180,NT1,False,164.6,False,Dist_NT1,3,0.000,n001,visual,2_7,29.24,29.24,15.4,15.4,cw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20203,1195,184.360627,180,NT1,False,197.9,False,Dist_NT1,16,35.025,n001,visual,2_7,-4.36,4.36,-17.9,17.9,ccw
20204,1196,186.362784,180,NT2,False,205.4,False,Dist_NT2,16,35.025,n001,visual,2_7,-6.36,6.36,-25.4,25.4,ccw
20205,1197,208.995104,180,T,False,192.4,False,Dist,26,35.025,n001,visual,2_7,-29.00,29.00,-12.4,12.4,ccw
20206,1198,181.640085,180,NT1,False,206.9,False,Dist_NT1,26,35.025,n001,visual,2_7,-1.64,1.64,-26.9,26.9,ccw


In [66]:
df.loc[(df['subject']=='l001') & (df['condition']=='1_0.2')  & (df['label_target']=='T') 
       & (df['brain_region']=='visual') & (df['TR']==0.000)  ]

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
75168,0,177.258102,180,T,False,164.0,False,Dist,0,0.0,l001,visual,1_0.2,2.74,2.74,16.0,16.0,cw
75171,3,164.948893,180,T,False,161.5,False,Dist,4,0.0,l001,visual,1_0.2,15.05,15.05,18.5,18.5,cw
75174,6,150.677208,180,T,True,349.0,True,Dist,7,0.0,l001,visual,1_0.2,29.32,29.32,-169.0,169.0,ccw
75177,9,194.419003,180,T,True,246.0,True,Dist,8,0.0,l001,visual,1_0.2,-14.42,14.42,-66.0,66.0,ccw
75180,12,180.749558,180,T,False,194.2,False,Dist,9,0.0,l001,visual,1_0.2,-0.75,0.75,-14.2,14.2,ccw
75183,15,185.266250,180,T,False,199.0,False,Dist,13,0.0,l001,visual,1_0.2,-5.27,5.27,-19.0,19.0,ccw
75186,18,175.374077,180,T,False,198.8,False,Dist,14,0.0,l001,visual,1_0.2,4.63,4.63,-18.8,18.8,ccw
75189,21,183.028689,180,T,True,357.0,True,Dist,18,0.0,l001,visual,1_0.2,-3.03,3.03,-177.0,177.0,ccw
75192,24,163.511397,180,T,True,75.0,True,Dist,19,0.0,l001,visual,1_0.2,16.49,16.49,105.0,105.0,cw
75195,27,158.817890,180,T,True,341.0,True,Dist,24,0.0,l001,visual,1_0.2,21.18,21.18,-161.0,161.0,ccw


In [67]:
df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ]

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5046,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,l001,1,1,1_1,0,visual,1_0.2
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2
5048,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,l001,1,1,1_1,2,visual,1_0.2
5049,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,l001,1,1,1_1,3,visual,1_0.2
5050,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,l001,1,1,1_1,4,visual,1_0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,2,3,0.2,11.8,334,228,69,142.0,214.9,40.7,...,142,228,214.9,l001,4,4,4_4,75,visual,1_0.2
5122,6,1,0.2,11.8,299,117,37,316.5,145.4,210.0,...,210,299,316.5,l001,4,4,4_4,76,visual,1_0.2
5123,8,3,0.2,11.8,334,123,62,233.0,110.0,39.1,...,233,123,110.0,l001,4,4,4_4,77,visual,1_0.2
5124,18,3,0.2,11.8,333,157,53,218.0,138.8,32.0,...,218,157,138.8,l001,4,4,4_4,78,visual,1_0.2


In [68]:
df_example  = df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ]
df_example[df_example['T'].values == df_example['T_alone'].values]

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2
5048,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,l001,1,1,1_1,2,visual,1_0.2
5049,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,l001,1,1,1_1,3,visual,1_0.2
5052,14,3,0.2,11.8,309,49,248,134.0,34.0,220.4,...,134,49,34.0,l001,1,1,1_1,6,visual,1_0.2
5053,18,3,0.2,11.8,301,52,233,110.0,35.8,203.5,...,110,52,35.8,l001,1,1,1_1,7,visual,1_0.2
5054,19,3,0.2,11.8,45,323,200,111.0,335.0,225.9,...,111,323,335.0,l001,1,1,1_1,8,visual,1_0.2
5057,8,3,0.2,11.8,306,146,42,242.0,156.3,66.8,...,242,146,156.3,l001,1,2,1_2,11,visual,1_0.2
5061,5,3,0.2,11.8,309,204,135,63.0,216.8,156.1,...,63,204,216.8,l001,1,3,1_3,15,visual,1_0.2
5062,6,3,0.2,11.8,140,49,331,225.0,33.3,308.6,...,225,49,33.3,l001,1,3,1_3,16,visual,1_0.2
5064,16,3,0.2,11.8,206,145,315,23.0,127.3,291.1,...,23,145,127.3,l001,1,3,1_3,18,visual,1_0.2


# Que pasa?

In [117]:
df_reconst = df.loc[(df['subject']=='n001') & (df['condition']=='1_0.2')  & (df['label_target']=='T') 
       & (df['brain_region']=='visual') & (df['TR']==0.000)  ]

In [118]:
df_reconst

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
15120,0,204.808354,180,T,False,164.0,False,Dist,0,0.0,n001,visual,1_0.2,-24.81,24.81,16.0,16.0,cw
15123,3,199.139792,180,T,False,161.5,False,Dist,4,0.0,n001,visual,1_0.2,-19.14,19.14,18.5,18.5,cw
15126,6,185.528899,180,T,True,349.0,True,Dist,7,0.0,n001,visual,1_0.2,-5.53,5.53,-169.0,169.0,ccw
15129,9,187.225924,180,T,True,246.0,True,Dist,8,0.0,n001,visual,1_0.2,-7.23,7.23,-66.0,66.0,ccw
15132,12,148.945889,180,T,False,194.2,False,Dist,9,0.0,n001,visual,1_0.2,31.05,31.05,-14.2,14.2,ccw
15135,15,189.189164,180,T,False,199.0,False,Dist,13,0.0,n001,visual,1_0.2,-9.19,9.19,-19.0,19.0,ccw
15138,18,211.385712,180,T,False,198.8,False,Dist,14,0.0,n001,visual,1_0.2,-31.39,31.39,-18.8,18.8,ccw
15141,21,174.250369,180,T,True,357.0,True,Dist,18,0.0,n001,visual,1_0.2,5.75,5.75,-177.0,177.0,ccw
15144,24,184.090092,180,T,True,75.0,True,Dist,19,0.0,n001,visual,1_0.2,-4.09,4.09,105.0,105.0,cw
15147,27,177.095055,180,T,True,30.0,True,Dist,24,0.0,n001,visual,1_0.2,2.90,2.90,150.0,150.0,cw


In [119]:
len(df_reconst)

27

In [ ]:
#### En los reconstructed, cuando voy a los "target" para un determinado subject, tengo 26 trials

In [120]:
df_behaviour = df_beh.loc[(df_beh['subject']=='n001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') ]

In [121]:
df_behaviour.head()

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
1035,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,n001,1,1,1_1,0,visual,1_0.2
1036,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,n001,1,1,1_1,1,visual,1_0.2
1037,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,n001,1,1,1_1,2,visual,1_0.2
1038,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,n001,1,1,1_1,3,visual,1_0.2
1039,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,n001,1,1,1_1,4,visual,1_0.2


In [122]:
len(df_behaviour)

96

In [131]:
df_behaviour.columns

Index(['Unnamed: 0', 'type', 'delay1', 'delay2', 'T', 'NT1', 'NT2', 'Dist',
       'Dist_NT1', 'Dist_NT2', 'distance_T_dist', 'cue', 'order', 'orient',
       'horiz_vertical', 'A_R', 'A_err', 'Abs_angle_error',
       'Error_interference', 'A_DC', 'A_DC_dist', 'Q_DC', 'A_DF', 'A_DF_dist',
       'Q_DF', 'A_DVF', 'Q_DVF', 'A_DVF_dist', 'Q_DVF_dist',
       'presentation_att_cue_time', 'presentation_target_time',
       'presentation_dist_time', 'presentation_probe_time', 'R_T',
       'trial_time', 'disp_time', 'T_alone', 'dist_alone', 'T_close',
       'dist_close', 'subject', 'session', 'run', 'session_run', 'new_index',
       'brain_region', 'condition'],
      dtype='object')

In [ ]:
actual_T = 'T'
actual_dist = 'Dist'
actual_response = 'A_R'
actual_Error = 'A_err'

In [123]:
## Veo que tengo 80 trials

In [124]:
### Solo estoy cogiendo una sesion????? Diría que sí! la primera, aunque hay algunos trials que bailan...

In [125]:
df_sess4 = df_behaviour.loc[df_behaviour['session']==1]
df_sess4.head()

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
1035,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,n001,1,1,1_1,0,visual,1_0.2
1036,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,n001,1,1,1_1,1,visual,1_0.2
1037,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,n001,1,1,1_1,2,visual,1_0.2
1038,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,n001,1,1,1_1,3,visual,1_0.2
1039,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,n001,1,1,1_1,4,visual,1_0.2


In [126]:
len(df_sess4)

35

In [127]:
### Podría ser.... Ver si encaja con la rotación de estos trials...

In [128]:
distance_to_ref =  df_sess4['T'].values- 180

        #
dist_180_ = df_sess4['Dist'].values - distance_to_ref
dist_180 = []
for n_dist in dist_180_:
    if n_dist<0:
        n_ = n_dist+360
        dist_180.append(n_)
    elif n_dist>360:
        n_ = n_dist-360
        dist_180.append(n_)
    else:
        dist_180.append(n_dist)
##
dist_180 = np.array(dist_180)

In [129]:
dist_180.sort()
dist_180

array([  5. ,  19. ,  20. ,  25. ,  30. ,  68. ,  70. ,  71. ,  75. ,
        92. ,  97. , 116. , 161.5, 162.9, 164. , 166.8, 166.9, 167.8,
       169.3, 190.5, 191.8, 194.1, 194.2, 198.8, 199. , 246. , 258. ,
       265. , 277. , 277. , 294. , 341. , 348. , 349. , 357. ])

In [130]:
sort__ = df_reconst.distractor_centered.values
sort__.sort()
sort__

array([  5. ,  20. ,  30. ,  70. ,  71. ,  75. ,  92. ,  97. , 116. ,
       161.5, 162.9, 164. , 166.8, 167.8, 191.8, 194.1, 194.2, 198.8,
       199. , 246. , 265. , 277. , 294. , 341. , 348. , 349. , 357. ])

In [96]:
df_behaviour.session_run.unique()

array(['1_1', '1_2', '1_3', '2_1', '2_2', '2_3', '3_1', '3_2', '3_4',
       '4_1', '4_2', '4_3', '4_4'], dtype=object)

In [98]:
for sess_run in df_behaviour.session_run.unique():
    df_behaviour.loc[df_behaviour['session_run']==sess_run].index.values 

In [103]:
df_behaviour.loc[df_behaviour['session_run']==sess_run].index.values

array([5120, 5121, 5122, 5123, 5124, 5125], dtype=int64)

In [105]:
training_indexes = []
testing_indexes =  []
for sess_run in df_behaviour.session_run.unique():
    wanted = df_behaviour.loc[df_behaviour['session_run']==sess_run].index.values 
    testing_indexes.append( wanted )
    #
    ## I do not trust the del  lines of other files, maybe this del inside a function in paralel is not removing the indexes, also you avoid going to lists to comeback
    all_indexes = df_behaviour.index.values
    other_indexes = all_indexes[~np.array([all_indexes[i] in wanted for i in range(len(all_indexes))])]  #take the ones that are not in wanted
    training_indexes.append( other_indexes )

In [107]:
testing_indexes

[array([5046, 5047, 5048, 5049, 5050, 5051, 5052, 5053, 5054, 5055, 5056],
       dtype=int64),
 array([5057, 5058], dtype=int64),
 array([5059, 5060, 5061, 5062, 5063, 5064, 5065, 5066, 5067, 5068],
       dtype=int64),
 array([5069, 5070, 5071, 5072, 5073, 5074, 5075, 5076, 5077, 5078, 5079],
       dtype=int64),
 array([5080, 5081, 5082, 5083], dtype=int64),
 array([5084, 5085], dtype=int64),
 array([5086, 5087, 5088, 5089, 5090, 5091, 5092], dtype=int64),
 array([5093, 5094, 5095, 5096], dtype=int64),
 array([5097, 5098, 5099], dtype=int64),
 array([5100, 5101, 5102, 5103, 5104, 5105, 5106, 5107, 5108], dtype=int64),
 array([5109, 5110, 5111, 5112], dtype=int64),
 array([5113, 5114, 5115, 5116, 5117, 5118, 5119], dtype=int64),
 array([5120, 5121, 5122, 5123, 5124, 5125], dtype=int64)]

In [115]:
testing_indexes[0]

array([5046, 5047, 5048, 5049, 5050, 5051, 5052, 5053, 5054, 5055, 5056],
      dtype=int64)

In [113]:
df_behaviour

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5046,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,l001,1,1,1_1,0,visual,1_0.2
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2
5048,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,l001,1,1,1_1,2,visual,1_0.2
5049,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,l001,1,1,1_1,3,visual,1_0.2
5050,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,l001,1,1,1_1,4,visual,1_0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,2,3,0.2,11.8,334,228,69,142.0,214.9,40.7,...,142,228,214.9,l001,4,4,4_4,75,visual,1_0.2
5122,6,1,0.2,11.8,299,117,37,316.5,145.4,210.0,...,210,299,316.5,l001,4,4,4_4,76,visual,1_0.2
5123,8,3,0.2,11.8,334,123,62,233.0,110.0,39.1,...,233,123,110.0,l001,4,4,4_4,77,visual,1_0.2
5124,18,3,0.2,11.8,333,157,53,218.0,138.8,32.0,...,218,157,138.8,l001,4,4,4_4,78,visual,1_0.2


In [76]:
df_behaviour

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5046,0,1,0.2,11.8,142,241,321,126.0,214.7,64.0,...,64,142,126.0,l001,1,1,1_1,0,visual,1_0.2
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2
5048,7,3,0.2,11.8,131,242,339,43.0,230.4,311.9,...,43,242,230.4,l001,1,1,1_1,2,visual,1_0.2
5049,8,3,0.2,11.8,313,39,243,153.0,22.4,215.4,...,153,39,22.4,l001,1,1,1_1,3,visual,1_0.2
5050,9,1,0.2,11.8,137,227,302,118.5,200.5,69.0,...,69,137,118.5,l001,1,1,1_1,4,visual,1_0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,2,3,0.2,11.8,334,228,69,142.0,214.9,40.7,...,142,228,214.9,l001,4,4,4_4,75,visual,1_0.2
5122,6,1,0.2,11.8,299,117,37,316.5,145.4,210.0,...,210,299,316.5,l001,4,4,4_4,76,visual,1_0.2
5123,8,3,0.2,11.8,334,123,62,233.0,110.0,39.1,...,233,123,110.0,l001,4,4,4_4,77,visual,1_0.2
5124,18,3,0.2,11.8,333,157,53,218.0,138.8,32.0,...,218,157,138.8,l001,4,4,4_4,78,visual,1_0.2


In [ ]:
df_example[df_example['T'].values == df_example['T_alone'].values]

In [11]:
df_beh.loc[(df_beh['subject']=='l001') & (df_beh['condition']=='1_0.2') & (df_beh['brain_region']=='visual') & (df_beh['new_index']==1) ]

,Unnamed: 0,type,delay1,delay2,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2,...,dist_alone,T_close,dist_close,subject,session,run,session_run,new_index,brain_region,condition
5047,4,3,0.2,11.8,312,42,232,113.0,29.4,206.6,...,113,42,29.4,l001,1,1,1_1,1,visual,1_0.2


In [12]:
df.loc[(df['subject']=='n001') & (df['condition']=='1_0.2') & (df['brain_region']=='visual') &  (df['new_index']==1)  &  (df['TR']==) ]

SyntaxError: invalid syntax (<ipython-input-12-48ae19705413>, line 1)

In [42]:
T=312
NT1=42
NT2=232

Dist=113.0
Dist_NT1=29.4
Dist_NT2=206.6


dif_t = T-180
T_1 = T - dif_t
Dist_1 = Dist - dif_t

if Dist_1<0:
    Dist_1=360 + Dist_1
elif Dist_1>360:
    Dist_1 = Dist_1-360

print(T_1, Dist_1)

dif_t = NT1-180
T_2 = NT1 - dif_t
Dist_2 = Dist_NT1 - dif_t

if Dist_2<0:
    Dist_2=360 + Dist_2
elif Dist_2>360:
    Dist_2 = Dist_2-360

print(T_2, Dist_2)

dif_t = NT2-180
T_3 = NT2 - dif_t
Dist_3 = Dist_NT2 - dif_t

if Dist_3<0:
    Dist_3=360 + Dist_3
elif Dist_3>360:
    Dist_3 = Dist_3-360

print(T_3, Dist_3)
    
    

180 341.0
180 167.4
180 154.6


In [38]:
360-(113-132)

379

In [18]:
df.loc[(df['subject']=='n001') & (df['condition']=='1_0.2') & (df['brain_region']=='visual') & (df['new_index']==3)  ]

,Unnamed: 0,decoded_angle,target_centered,label_target,corresp_isolated,distractor_centered,corresp_isolated_distractor,label_distractor,new_index,TR,subject,brain_region,condition,error,error_abs,distractor_distance,distractor_distance_abs,sign_dist
15180,60,184.964691,180,T,True,20.0,True,Dist,3,0.000,n001,visual,1_0.2,-4.96,4.96,160.0,160.0,cw
15181,61,138.026423,180,NT1,False,163.4,False,Dist_NT1,3,0.000,n001,visual,1_0.2,41.97,41.97,16.6,16.6,cw
15182,62,159.501153,180,NT2,False,152.4,False,Dist_NT2,3,0.000,n001,visual,1_0.2,20.50,20.50,27.6,27.6,cw
15261,141,174.926297,180,T,True,20.0,True,Dist,3,2.335,n001,visual,1_0.2,5.07,5.07,160.0,160.0,cw
15262,142,155.457397,180,NT1,False,163.4,False,Dist_NT1,3,2.335,n001,visual,1_0.2,24.54,24.54,16.6,16.6,cw
15263,143,181.817055,180,NT2,False,152.4,False,Dist_NT2,3,2.335,n001,visual,1_0.2,-1.82,1.82,27.6,27.6,cw
15342,222,158.614798,180,T,True,20.0,True,Dist,3,4.670,n001,visual,1_0.2,21.39,21.39,160.0,160.0,cw
15343,223,173.118246,180,NT1,False,163.4,False,Dist_NT1,3,4.670,n001,visual,1_0.2,6.88,6.88,16.6,16.6,cw
15344,224,167.760673,180,NT2,False,152.4,False,Dist_NT2,3,4.670,n001,visual,1_0.2,12.24,12.24,27.6,27.6,cw
15423,303,183.327886,180,T,True,20.0,True,Dist,3,7.005,n001,visual,1_0.2,-3.33,3.33,160.0,160.0,cw


In [16]:
1296/3/27

16.0